# Introduction

This notebook demonstrates the preprocessing of raw shallot price data to prepare it for time series forecasting using AutoML tools.

In [99]:
from google.cloud import storage

import pandas as pd
import numpy as np
import csv
import os
import io

In [100]:
# Helper functions

def download_csv_from_gcs(bucket, file_name,
                          date_columns=None, col_names=None):
    """ A function to download dataset from GCS. """

    blob = bucket.blob(file_name)
    data = blob.download_as_text()
    df = pd.read_csv(io.StringIO(data),
                     parse_dates=date_columns,
                     usecols=col_names)
    return df

def upload_csv_to_gcs(df, bucket, file_path, sep=','):
    """ A function to upload dataset to GCS. """

    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

    blob = bucket.blob(file_path)
    blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')

    print(f"Successfully uploaded: '{blob.name}' to '{bucket.name}'")

In [102]:
# Create a client GCS and get the specified bucket
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)

In [103]:
# Download the dataset from GCS
prices_df = download_csv_from_gcs(bucket, DATA_RAW_CSV, date_columns=["date"])

In [104]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             2404 non-null   datetime64[ns]
 1   surabayakota     2404 non-null   int64         
 2   malangkota       2404 non-null   int64         
 3   kedirikota       2404 non-null   int64         
 4   jemberkab        2404 non-null   int64         
 5   bangkalankab     2404 non-null   int64         
 6   banyuwangikab    2404 non-null   int64         
 7   blitarkab        2404 non-null   int64         
 8   bojonegorokab    2404 non-null   int64         
 9   bondowosokab     2404 non-null   int64         
 10  gresikkab        2404 non-null   int64         
 11  jombangkab       2404 non-null   int64         
 12  kedirikab        2404 non-null   int64         
 13  lamongankab      2404 non-null   int64         
 14  lumajangkab      2404 non-null   int64  

In [105]:
prices_df.set_index("date", inplace=True)

In [106]:
# For our analysis, we will focus on two regions in East Java: Mojokerto and Surabaya
selected_regions = ["mojokertokab", "surabayakota"]
prices_df = prices_df[selected_regions]
prices_df = prices_df.rename(columns={"mojokertokab": "mojokerto",
                                      "surabayakota": "surabaya"})

## Dataset Overview

In [107]:
prices_df.head()

,mojokerto,surabaya
date,,
2018-01-01,14750,20000
2018-01-02,14750,19800
2018-01-03,14750,20600
2018-01-04,14750,20600
2018-01-05,14750,20600


In [108]:
# describe the statistics
prices_df.agg(["min", "max", "median", "mean", "std"]).T

,min,max,median,mean,std
mojokerto,12000.0,68000.0,24250.0,25136.561980,7089.663609
surabaya,14600.0,64400.0,29000.0,29622.760399,7465.056849


In [109]:
prices_df.isna().sum()

,0
mojokerto,0
surabaya,0


As we can see from the statistics above, there are no missing values in this dataset.

Next, we ensure that the date range is complete, with no dates skipped.

In [110]:
prices_df.index.min(), prices_df.index.max()

(Timestamp('2018-01-01 00:00:00'), Timestamp('2024-07-31 00:00:00'))

In [111]:
ds_start = "2018-01-01"
ds_end = "2024-07-31"

complete_date = pd.date_range(start=ds_start, end=ds_end, freq="D", name="date")
prices_df = prices_df.reindex(complete_date)

In [112]:
prices_df.isna().sum()

,0
mojokerto,0
surabaya,0


## Splitting Data into Train, Validation, and Test Sets

In [113]:
# Devide data based on the timestamp
test_end = "2024-07-31"

val_end = pd.to_datetime(test_end) - pd.to_timedelta(15, unit="D")
train_end = pd.to_datetime(val_end) - pd.to_timedelta(15, unit="D")

val_start = pd.to_datetime(train_end) + pd.to_timedelta(1, unit="D")
test_start = pd.to_datetime(val_end) + pd.to_timedelta(1, unit="D")

In [114]:
print(f"Train => {ds_start} to {train_end.strftime('%Y-%m-%d')}")
print(f"Valid => {val_start.strftime('%Y-%m-%d')} to {val_end.strftime('%Y-%m-%d')}")
print(f"Test  => {test_start.strftime('%Y-%m-%d')} to {test_end}")

Train => 2018-01-01 to 2024-07-01
Valid => 2024-07-02 to 2024-07-16
Test  => 2024-07-17 to 2024-07-31


In [115]:
prices_df.loc[prices_df.index <= train_end, "data_split"] = "TRAIN"

prices_df.loc[(prices_df.index >= val_start) &
              (prices_df.index <= val_end), "data_split"] = "VALIDATE"

prices_df.loc[(prices_df.index >= test_start) &
              (prices_df.index <= test_end), "data_split"] = "TEST"

In [116]:
# Format the dataset into a long dataset to merge the regions into a column
df = prices_df.reset_index().melt(id_vars=["date", "data_split"],
                               value_vars=["mojokerto", "surabaya"],
                               var_name="regions",
                               value_name="price")

In [117]:
df.groupby(["data_split", "regions"]).head(2)

,date,data_split,regions,price
0,2018-01-01,TRAIN,mojokerto,14750
1,2018-01-02,TRAIN,mojokerto,14750
2374,2024-07-02,VALIDATE,mojokerto,26500
2375,2024-07-03,VALIDATE,mojokerto,24250
2389,2024-07-17,TEST,mojokerto,22412
2390,2024-07-18,TEST,mojokerto,22412
2404,2018-01-01,TRAIN,surabaya,20000
2405,2018-01-02,TRAIN,surabaya,19800
4778,2024-07-02,VALIDATE,surabaya,29666
4779,2024-07-03,VALIDATE,surabaya,29333


In [118]:
upload_csv_to_gcs(df, bucket, "shallot_dataset.csv")

Successfully uploaded: 'shallot_dataset.csv' to 'commodity_prices_automl'
